Below is the basic example of Dag

In [ ]:
from airflow import DAG
from airflow.utils.dates import days_ago
from airflow.operators.bash_operator import BashOperator

dag = DAG(
        dag_id = "testing_dag",
        schedule_interval = "@daily",
        start_date = days_ago(1)
        )

task1 = BashOperator(
        task_id = "task1",
        bash_command = "echo hello",
        dag = dag
        )

### How to set precendencies between different tasks

Below code will run the tasks(1,2,3) in series

In [ ]:
from airflow import DAG
from airflow.utils.dates import days_ago
from airflow.operators.bash_operator import BashOperator

dag = DAG(
        dag_id = "testing_dag",
        schedule_interval = "@daily",
        start_date = days_ago(1)
        )

task1 = BashOperator(
        task_id = "task1",
        bash_command = "echo hello",
        dag = dag
        )


task2 = BashOperator(
        task_id = "task2",
        bash_command = "echo task2",
        dag = dag
        )

task3 = BashOperator(
        task_id = "task3",
        bash_command = "echo task3",
        dag = dag
        )

task1 >> task2 >> task

If you want to run the two tasks in parallel then you've to put them in a list.

Below will run the task2 and task3 in parallel after task1

```python
task1 >> [task2, task3]
```

### Lifecycle of a task

* Scheduled - Up for execution or ready for execution
* Skipped - for ex - I'm reading a file and if the file has something then schedule it else skip it and schedule the next task
* upstream_failed - if the parent task is failed then a task will be marked as upstream_failed.
* failed - Task that is failed
* Running - Currently running task
* Success - Task executed successfully
* up_for_retry - failed task is getting rerun if retry>1 is provided

```python
task1 >> [task2, task3]
```
Let's say I want to do something like following:

```
if task1 == success:
  then:
    execute task2
  else:
    execute task3
```

In the above cases we've to use the trigger_rule, there are various trigger rules:

Default trigger_rule is all_success

* all_success - all the parents are success
* all_failed - all the parents are failed
* all_done - All the parents have been attempted to be executed or triggerd irrespective of fail or success
* one_failed - `[t1, t2] >> t3` In this case either t1 or t2 gets failed then only t3 will be executed.
* one_success - `[t1, t2] >> t3` In this case either t1 or t2 gets successful then only t3 will be executed.
* none_failed - all of the parents are either success or skipped
* none_failed_or_skipped - atleast one parent has ssucceeded
* none_skipped - none of the parents are skipped
* dummy - on it own will

Below is the code for scenerio `all_failed` on task:

```python
from airflow import DAG
from airflow.utils.dates import days_ago
from airflow.operators.bash_operator import BashOperator

dag = DAG(
        dag_id = "testing_dag",
        schedule_interval = "@daily",
        start_date = days_ago(1)
        )

task1 = BashOperator(
        task_id = "task1",
        bash_command = "echo hello && exit 1",
        dag = dag
        )


task2 = BashOperator(
        task_id = "task2",
        bash_command = "echo task2",
        dag = dag
        )

task3 = BashOperator(
        task_id = "task3",
        bash_command = "echo task3",
        dag = dag,
        trigger_rule = "all_failed"
        )

task1 >> [task2, task3]
```

### Creating custom Operators and HTTPsensors

Please refer to the below code:

```python
from airflow import DAG
from airflow.utils.dates import days_ago
from airflow.operators.bash_operator import BashOperator
from airflow.models import BaseOperator
from airflow.utils.decorators import apply_defaults
from airflow.sensors.http_sensor import HttpSensor
from datetime import timedelta


"""
Below code is responsible for creating your own operator
"""
class MyOperator(BaseOperator):
    @apply_defaults
    def __init__(self, name, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.name = name

    def execute(self, context):
        message = f"Hello {self.name}"
        print(message)
        return message

dag = DAG(
        dag_id = "testing_dag",
        schedule_interval = "@daily",
        start_date = days_ago(1)
        )

task1 = BashOperator(
        task_id = "task1",
        bash_command = "echo hello && exit 1",
        dag = dag
        )


task2 = BashOperator(
        task_id = "task2",
        bash_command = "echo task2",
        dag = dag
        )

task3 = BashOperator(
        task_id = "task3",
        bash_command = "echo task3",
        dag = dag,
        trigger_rule = "all_failed"
        )

task4 = MyOperator(
        name = "Tushar",
        task_id = "task4",
        dag =dag,
        trigger_rule = "one_success"
        )
"""
creating a http sensor, It will check the service running
on a host and a particular port with endpoint as '/'
make sure to configure the my_http_con in airflow ui connections
"""
sensor = HttpSensor(
        task_id = "sensor",
        endpoint = "/",
        http_conn_id = "my_http_con",
        dag = dag,
        retries = 20,
        retry_delay = timedelta(seconds=20)
        )

sensor >> task1 >> [task2, task3] >> task4
```
